# Use opencv-python to create an animation
### [Opencv](https://pypi.org/project/opencv-python/) 

In [ ]:
pip install opencv-python

In [ ]:
import cv2

# Background

In [ ]:
directory   = '/srv/scratch/z3533156'
from IPython.display import Image
Image(directory+"/PUG/Topography.png")

In [ ]:
Image(directory+"/PUG/EAC_Scenarios.png")

# Import packages

In [ ]:
import os
import cmaps
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from netCDF4 import Dataset
from datetime import date
from mpl_toolkits.basemap import Basemap

# Read grid information and dye concentration data

In [ ]:
# Read longitude, latitude and depth.
HSM_grid=Dataset(directory + '/HSM/coastal_raw_rx018.nc','r')
lon=HSM_grid.variables['lon_rho'][10:200,0:40]
lat=HSM_grid.variables['lat_rho'][10:200,0:40]
dep=HSM_grid.variables['h'][10:200,0:40]
# EAC Scenario1
Scenario1_avg=Dataset(directory + '/HSM/Scenario1/outer_avg_02.nc','r')
Scenario1_u=Scenario1_avg.variables['u_eastward'][:,29,10:200,0:40]
Scenario1_v=Scenario1_avg.variables['v_northward'][:,29,10:200,0:40]
Scenario1_dye=Scenario1_avg.variables['dye_01'][:,29,10:200,0:40]
Scenario1_time=np.squeeze(Scenario1_avg.variables['ocean_time'][:]/86400)
# EAC Scenario2
Scenario2_avg=Dataset(directory + '/HSM/Scenario2/outer_avg_02.nc','r')
Scenario2_u=Scenario2_avg.variables['u_eastward'][:,29,10:200,0:40]
Scenario2_v=Scenario2_avg.variables['v_northward'][:,29,10:200,0:40]
Scenario2_time=np.squeeze(Scenario2_avg.variables['ocean_time'][:]/86400)
Scenario2_dye=Scenario2_avg.variables['dye_01'][:,29,10:200,0:40]
# EAC Scenario3
Scenario3_avg=Dataset(directory + '/HSM/Scenario3/outer_avg_02.nc','r')
Scenario3_u=Scenario3_avg.variables['u_eastward'][:,29,10:200,0:40]
Scenario3_v=Scenario3_avg.variables['v_northward'][:,29,10:200,0:40]
Scenario3_dye=Scenario3_avg.variables['dye_01'][:,29,10:200,0:40]
Scenario3_time=np.squeeze(Scenario3_avg.variables['ocean_time'][:]/86400)

In [ ]:
# Total 180 sample size in 15 days
np.shape(Scenario1_time)

# Import functions

In [ ]:
def serial_day_to_datestring(day,epochY=1990,epochm=1,epochd=1,epochH=0,epochM=0):
    import datetime
    corrected_date = datetime.datetime(epochY,epochm,epochd,epochH,epochM) + datetime.timedelta(day)
    return corrected_date.strftime("%Y-%m-%d %H:%M")  

In [ ]:
serial_day_to_datestring(Scenario1_time[1])

# Set the figure parameters

In [ ]:
## Velocity parameters
m     = Basemap(projection='merc',llcrnrlat=np.nanmin(lat),urcrnrlat= np.nanmax(lat),
              llcrnrlon=np.nanmin(lon),urcrnrlon=np.nanmax(lon),resolution='h')
x,y = m(lon,lat)
yy = np.arange(0, y.shape[0], 5)
xx = np.arange(0, x.shape[1], 5)
points = np.meshgrid(yy, xx)
point_index=tuple(points)
point_x=x[point_index]
point_y=y[point_index]
y_quiver = 0.88
cur_var  = 0.5
str_var  = ['0.5 m/s']
## River locations
lon1  = np.array([lon[146,3],lon[156,3]])
lat1  = np.array([lat[146,3],lat[156,3]])
x1,y1 = m(lon1,lat1)
rx=np.zeros(5)
ry=np.zeros(5)
rtx=np.zeros(5)
rty=np.zeros(5)
rivers  = ['HR','SH','BB','PH','PK']
rx[0],ry[0] = m(151.365721, -33.579100) # 1 (HR) - Hunter River   
rx[1],ry[1] = m(151.246587, -33.862418) # 2 (SH) - Sydney Harbour  
rx[2],ry[2] = m(151.237908, -34.010494) # 3 (BB) - Botany Bay     
rx[3],ry[3] = m(151.189933, -34.073633) # 4 (PH) - Port Hacking   
rx[4],ry[4] = m(150.908340, -34.470658) # 5 (PK) - Port Kembla    
rtx[0],rty[0] = m(151.365721-0.1, -33.579100-0.02)  # 1 (HR) - Hunter River   
rtx[1],rty[1] = m(151.246587-0.1, -33.862418-0.02)  # 2 (SH) - Sydney Harbour 
rtx[2],rty[2] = m(151.237908-0.1, -34.010494-0.02)  # 3 (BB) - Botany Bay  
rtx[3],rty[3] = m(151.189933-0.1, -34.073633-0.02)  # 4 (PH) - Port Hacking
rtx[4],rty[4] = m(150.908340-0.1, -34.470658-0.02)  # 5 (PK) - Port Kembla    
## Figure parameters
fig_ratio=0.9
gs=gridspec.GridSpec(1,3)
labelfont=40
padspacescale=20
labelpadscale=5
marker_size=20
linefont = 8
scale  = 1.18
cmaps1 = cmaps.WhiteBlueGreenYellowRed
levels1 = np.linspace(0,1,100)
tick_marks1 = np.linspace(0,1,11)
mx1,my1 = m(150.81, -33.39)
labels = ['(a)Scenario1','(b)Scenario2','(c)Scenario3']

# Plot the dye concentration for three EAC scenarios

In [ ]:
len=3 # 180
for j in range(len):
    fig = plt.figure(figsize=(30, 30))
    for i in range(3):
        ax = fig.add_subplot(gs[i])
        l, b, w, h = ax.get_position().bounds
        l=l-0.05
        ax.set_position([l, b, scale*w, scale*h]) 
        m   = Basemap(projection='merc',llcrnrlat=np.nanmin(lat),urcrnrlat= np.nanmax(lat),llcrnrlon=np.nanmin(lon),urcrnrlon=np.nanmax(lon),resolution='h')
        m.drawmeridians(np.arange(150,152, 0.5),labels=[0,0,0,1],linewidth=0.2*linefont,dashes=[2,2],color='.5',fontsize=1.0*labelfont)
        m.drawparallels(np.arange(-35,-33, 0.2),labels=[0,0,0,0],linewidth=0.2*linefont,dashes=[2,2],color='.5',fontsize=1.0*labelfont)    
        m.drawstates(linewidth=linefont, linestyle='dashed', color='k', antialiased=1, ax=None, zorder=None)
        m.drawcoastlines(color='xkcd:grey',  linewidth=0.2*linefont)
        m.drawmapboundary(color='0.1', linewidth=0.2*linefont)
        m.fillcontinents(color='0.95', lake_color='white')
        # EAC Scenario1
        if i<1:
            scenario_time=Scenario1_time
            scenario_con=Scenario1_dye
            u_Var=Scenario1_u
            v_Var=Scenario1_v
        # EAC Scenario2
        elif i<2:
            scenario_time=Scenario2_time
            scenario_con=Scenario2_dye
            u_Var=Scenario2_u
            v_Var=Scenario2_v
        # EAC Scenario3
        else:
            scenario_time=Scenario3_time
            scenario_con=Scenario3_dye
            u_Var=Scenario3_u
            v_Var=Scenario3_v
        u=u_Var[j,:,:]
        v=v_Var[j,:,:]
        CB = m.contourf(x, y, scenario_con[j,:,:],cmap=cmaps1,levels=levels1,origin='lower',extend='both') 
        CS = m.contour(x, y, dep,np.linspace(99,101,10),linewidths=0.5*linefont,linestyles='solid',colors='xkcd:grey')
        Q  = m.quiver(point_x,point_y,u[point_index],v[point_index],color='xkcd:grey',width=0.004,scale=5.0)
        qk = plt.quiverkey(Q, 0.15, 0.81, cur_var, str_var[0],labelpos='N',color='xkcd:grey',linewidth=linefont*0.4,fontproperties={'size': labelfont}) 
        m.plot(x1, y1,linewidth=linefont, linestyle='solid', color='xkcd:black')
        # Labels the rivers
        for k in range(5):
            if k>0:
                m.plot(rx[k], ry[k], marker='.',color='xkcd:black', markersize=marker_size)
            plt.text(rtx[k], rty[k],rivers[k],color='xkcd:black', fontsize=labelfont)
        if i==0:
            m.drawparallels(np.arange(-35,-33, 0.2),labels=[1,0,0,0],linewidth=0.3*linefont,dashes=[2,2],color='.5',fontsize=1.0*labelfont)    
        ax.spines['left'].set_linewidth(labelpadscale)
        ax.spines['top'].set_linewidth(labelpadscale)
        ax.spines['right'].set_linewidth(labelpadscale)
        ax.spines['bottom'].set_linewidth(labelpadscale)
        plt.title(labels[i], fontsize=1.0*labelfont,loc='left',pad=0)
        plt.text(mx1,my1,serial_day_to_datestring(scenario_time[j]), color='black', fontsize=1.0*labelfont)
    cbaxes1  = fig.add_axes([0.085, 0.33, 0.8, 0.01])
    cb1 = plt.colorbar(CB,orientation='horizontal',cax = cbaxes1)
    cb1.set_ticks(tick_marks1)
    cb1.ax.tick_params(labelsize=1.0*labelfont)
    cb1.set_label('(Dye concentration)', fontsize=1.0*labelfont,labelpad=0)    
    fig.savefig(directory+'/PUG/Scenario_dye_HSM_'+str(j+1)+'.png')
#     plt.close(fig)

# Create an animation to show the evolution of dye concentration

In [ ]:
# Read image files
img_array = []
for i in range(30):
    filename=(directory+'/PUG/Scenario_dye_HSM_'+str(i+1)+'.png')
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

## Output the animation   
# cv2.VideoWriter(filename, fourcc, fps, framesize, isColor)
out = cv2.VideoWriter(directory+'/PUG/Scenario_dye_HSM.avi',cv2.VideoWriter_fourcc(*'DIVX'), 5, size) 
for i in range(np.size(img_array,0)):
    out.write(img_array[i])
out.release()
# os.system('rm '+directory+'/PUG/Scenario_dye_HSM_*.png')

## Python cv2.VideoWriter() Examples
### [cv2.VideoWriter](https://www.programcreek.com/python/example/72134/cv2.VideoWriter) 